**Gender Behaviour Analysis**

# Analisis Perilaku Pengguna: Gender, Swipe Right Ratio, dan Likes

Notebook ini merangkum hasil analisis eksplorasi data terkait perilaku pengguna aplikasi
dating berdasarkan gender. Analisis berfokus pada:

1. Perbedaan **Swipe Right Ratio** antar **Gender**.
2. Jumlah **Likes Received** berdasarkan **Gender**.
3. Hubungan antara **App Usage Time(minutes)** dengan **Likes Receive**.
4. Persebaran **Sexual Orientation** berdasar **Gender** dan analisa Chi square untuk melihat korelasinya.

Dataset berasal dari database `DatingSQL` dengan total ~50 ribu baris data.


# Import dan Load Data

Pada bagian ini akan digunakan untuk mengimpor semua library dan juga memberikan ringkasan statistik tahap awal.

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import f_oneway

# Import fungsi dari utils & script EDA
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from utils.db_connection import *
from utils.cleaning import *

# Load data
df_query = load_query('* FROM user_data')
pd.set_option('display.max_colwidth', 12)
pd.set_option('display.precision', 2)
df_query.head()


c:\Users\user\Downloads\DataRole\DSPy\utils\db_connection.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM user_data", conn)


,gender,sexual_orientation,location_type,income_bracket,education_level,interest_tags,app_usage_time_min,app_usage_time_label,swipe_right_ratio,swipe_right_label,likes_received,mutual_matches,profile_pics_count,bio_length,message_sent_count,emoji_usage_rate,last_active_hour,swipe_time_of_day,match_outcome
0,Prefer N...,Gay,Urban,High,Bachelor’s,"Fitness,...",52,Moderate,0.60,Optimistic,173,23,4,44,75,0.36,13,Early Mo...,Mutual M...
1,Male,Bisexual,Suburban,Upper-Mi...,No Forma...,Language...,279,Extreme ...,0.56,Optimistic,107,7,3,301,35,0.42,0,Morning,Chat Ign...
2,Non-binary,Pansexual,Suburban,Low,Master’s,"Movies, ...",49,Moderate,0.41,Optimistic,91,27,2,309,33,0.41,1,After Mi...,Date Hap...
3,Genderfluid,Gay,Metro,Very Low,Postdoc,"Coding, ...",185,Extreme ...,0.32,Balanced,147,6,5,35,5,0.07,21,Morning,No Action
4,Male,Bisexual,Urban,Middle,Bachelor’s,Clubbing...,83,High,0.32,Balanced,94,11,1,343,34,0.11,22,After Mi...,One-side...


In [12]:
print("Deskripsi singkat statistik dari dataset:\n")
df_query.describe(include='all')

Deskripsi singkat statistik dari dataset:



,gender,sexual_orientation,location_type,income_bracket,education_level,interest_tags,app_usage_time_min,app_usage_time_label,swipe_right_ratio,swipe_right_label,likes_received,mutual_matches,profile_pics_count,bio_length,message_sent_count,emoji_usage_rate,last_active_hour,swipe_time_of_day,match_outcome
count,50000,50000,50000,50000,50000,50000,50000.00,50000,50000.00,50000,50000.00,50000.00,50000.00,50000.00,50000.00,50000.00,50000.00,50000,50000
unique,6,8,6,7,9,40206,NaN,7,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,10
top,Female,Straight,Remote Area,High,Bachelor’s,"Fitness,...",NaN,Extreme ...,NaN,Optimistic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,After Mi...,One-side...
freq,8384,6326,8519,7309,5646,6,NaN,20140,NaN,26873,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8524,5112
mean,NaN,NaN,NaN,NaN,NaN,NaN,149.91,NaN,0.50,NaN,99.53,13.87,2.99,250.17,50.07,0.29,11.52,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,86.99,NaN,0.20,NaN,58.00,9.11,2.00,144.80,29.17,0.16,6.92,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,74.00,NaN,0.37,NaN,49.00,6.00,1.00,125.00,25.00,0.16,5.00,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,150.00,NaN,0.50,NaN,100.00,13.00,3.00,250.00,50.00,0.27,12.00,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,225.00,NaN,0.64,NaN,150.00,22.00,5.00,376.00,75.00,0.39,18.00,NaN,NaN


**Jumlah Baris yang ada duplikat**

In [13]:
print("Jumlah baris duplikat:", df_query.duplicated().sum())

Jumlah baris duplikat: 0


# Analisis perilaku yang berkaitan dengan gender

**Heatmap gender dan Uji Chi Square**

Pada tahap analisis ini akan dibuat analisa apakah ada ketimpangan jumlah sample per gender